In [8]:
import pandas as pd
import numpy as np
import redis

In [9]:
hostname = 'redis-18184.c11.us-east-1-2.ec2.redns.redis-cloud.com'
portnumber = 18184
password = '1KQWjPR6PgkXXFfTIgh4UnA1yLz8yOyo'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)


In [10]:
r.ping()

True

In [11]:
name_role_to_batch = {

    'Aung Khant Kyaw@Student': 86,
    'Zin Lin Htet@Student' : 78,
    'Aung Paing Oo@Student' : 79,
    'Thin Yadanar Oo@Student' : 82,
    'Hsu Myat Noe@Student' : 86,
    'Tun Tun Nyein@Student' : 29, 
    'Yin Kyi Phyu@Student' : 86,
    'Win Htut Naing@Student' : 82,
    'Su Lei Aung@Student':83,
    'Aye Myat Phoo Khine@Student' : 83,
    "Ye' Thein Htut@Student" : 82,
    'Htan Lin Han@Student' : 82,
    'Wuttyi Oo@Student' : 86, 
    'May Moe Pwint Phyu@Student' : 83, 
    'Aye Chan Myat Phyoe@Student' : 86,
    'Dr. Moe Arkar Lwin@Teacher' : 'None',
    
    # Add all other mappings here
}

In [13]:

def migrate_old_data(name_role_to_batch):
    for old_key, batch in name_role_to_batch.items():
        if '@' in old_key and old_key.count('@') == 1:  # Old format check
            name, role = old_key.split('@')
            new_key = f'{name}@{role}@{batch}'
            
            # Retrieve the data associated with the old key
            data = r.hgetall(old_key)
            
            if data:
                # Save data to the new key format
                r.hset(new_key, mapping=data)
                
                # Optionally delete the old key
                r.delete(old_key)
                
                print(f"Migrated {old_key} to {new_key}")
            else:
                print(f"No data found for {old_key}")

# Run migration with the provided mapping
migrate_old_data(name_role_to_batch)


No data found for Aung Khant Kyaw@Student
No data found for Zin Lin Htet@Student
No data found for Aung Paing Oo@Student
No data found for Thin Yadanar Oo@Student
No data found for Hsu Myat Noe@Student
No data found for Tun Tun Nyein@Student
No data found for Yin Kyi Phyu@Student
No data found for Win Htut Naing@Student
No data found for Su Lei Aung@Student
No data found for Aye Myat Phoo Khine@Student
No data found for Ye' Thein Htut@Student
No data found for Htan Lin Han@Student
No data found for Wuttyi Oo@Student
No data found for May Moe Pwint Phyu@Student
No data found for Aye Chan Myat Phyoe@Student
No data found for Dr. Moe Arkar Lwin@Teacher


In [14]:
def get_field_names(key):
    # Retrieve field names (i.e., keys) from the hash
    field_names = r.hkeys(key)
    return [field_name.decode() for field_name in field_names]

# Example usage
key = 'info:myanmar'
field_names = get_field_names(key)
print("Field Names in 'info:myanmar' key:")
for field_name in field_names:
    print(field_name)

Field Names in 'info:myanmar' key:
Aye Chan Myat Phyoe@Student
Kyaw Kyaw@Student
Aung Khant Kyaw@Student
Zin Lin Htet@Student
Su Lei Aung@Student
Ye' Thein Htut@Student
Dr. Moe Arkar Lwin@Teacher
Win Htut Naing@Student
Hsu Myat Noe@Student
Tun Tun Nyein@Teacher
Aung Paing Oo@Student
Aye Myat Phoo Khine@Student
May Moe Pwint Phyu@Student
Htan Lin Han@Student
Bo Bo Min@Student
Thin Yadanar Oo@Student
Yin Kyi Phyu@Student
Wuttyi Oo@Student
Bo Bo Min@Student@78


In [15]:
def update_field_names_with_batch(key, batch_dict):
    # Retrieve field names (i.e., keys) from the hash
    field_names = r.hkeys(key)
    for field_name_bytes in field_names:
        field_name = field_name_bytes.decode()
        if field_name in batch_dict:
            batch = batch_dict[field_name]
            name, role = field_name.split('@')
            new_field_name = f"{name}@{role}@{batch}"
            
            # Get the value associated with the old field name
            value = r.hget(key, field_name)
            
            # Set the value associated with the new field name
            r.hset(key, new_field_name, value)
            
            # Optionally delete the old field name
            r.hdel(key, field_name)
            
            print(f"Updated field name from '{field_name}' to '{new_field_name}'")
        else:
            print(f"No batch number found for field '{field_name}'. Skipping...")

# Example usage

update_field_names_with_batch(key, name_role_to_batch)

Updated field name from 'Aye Chan Myat Phyoe@Student' to 'Aye Chan Myat Phyoe@Student@86'
No batch number found for field 'Kyaw Kyaw@Student'. Skipping...
Updated field name from 'Aung Khant Kyaw@Student' to 'Aung Khant Kyaw@Student@86'
Updated field name from 'Zin Lin Htet@Student' to 'Zin Lin Htet@Student@78'
Updated field name from 'Su Lei Aung@Student' to 'Su Lei Aung@Student@83'
Updated field name from 'Ye' Thein Htut@Student' to 'Ye' Thein Htut@Student@82'
Updated field name from 'Dr. Moe Arkar Lwin@Teacher' to 'Dr. Moe Arkar Lwin@Teacher@None'
Updated field name from 'Win Htut Naing@Student' to 'Win Htut Naing@Student@82'
Updated field name from 'Hsu Myat Noe@Student' to 'Hsu Myat Noe@Student@86'
No batch number found for field 'Tun Tun Nyein@Teacher'. Skipping...
Updated field name from 'Aung Paing Oo@Student' to 'Aung Paing Oo@Student@79'
Updated field name from 'Aye Myat Phoo Khine@Student' to 'Aye Myat Phoo Khine@Student@83'
Updated field name from 'May Moe Pwint Phyu@Studen

In [18]:
# Define the name_role_to_batch dictionary
name_role_to_batch = {
    'Aung Khant Kyaw@Student': 86,
    'Zin Lin Htet@Student': 78,
    'Aung Paing Oo@Student': 79,
    'Thin Yadanar Oo@Student': 82,
    'Hsu Myat Noe@Student': 86,
    'Tun Tun Nyein@Student': 29,
    'Yin Kyi Phyu@Student': 86,
    'Win Htut Naing@Student': 82,
    'Su Lei Aung@Student': 83,
    'Aye Myat Phoo Khine@Student': 83,
    "Ye' Thein Htut@Student": 82,
    'Htan Lin Han@Student': 82,
    'Wuttyi Oo@Student': 86,
    'May Moe Pwint Phyu@Student': 83,
    'Aye Chan Myat Phyoe@Student': 86,
    'Dr. Moe Arkar Lwin@Teacher': 'None'
    # Add all other mappings here
}




In [23]:
name_role_to_batch = {
    'Aung Khant Kyaw@Student': 86,
    'Zin Lin Htet@Student': 78,
    'Aung Paing Oo@Student': 79,
    'Thin Yadanar Oo@Student': 82,
    'Hsu Myat Noe@Student': 86,
    'Tun Tun Nyein@Student': 29,
    'Yin Kyi Phyu@Student': 86,
    'Win Htut Naing@Student': 82,
    'Su Lei Aung@Student': 83,
    'Aye Myat Phoo Khine@Student': 83,
    "Ye' Thein Htut@Student": 82,
    'Htan Lin Han@Student': 82,
    'Wuttyi Oo@Student': 86,
    'May Moe Pwint Phyu@Student': 83,
    'Aye Chan Myat Phyoe@Student': 86,
    'Dr. Moe Arkar Lwin@Teacher': 'None'
    # Add all other mappings here
}

def func(name_role_to_batch):
    att_logs = r.lrange('attendance:logs',0,-1)
    update_logs = []
    for log in att_logs:
        log_str = log.decode()
        name_role,timestamp = log_str.split('@',1)
        print(name_role)
        print(timestamp)

func(name_role_to_batch)

Bo Bo Min
Student@2024-05-13 13:22:28.389529
Bo Bo Min
Student@2024-05-16 11:29:28.776530
Zin Lin Htet
Student@2024-05-16 11:30:21.544281
Zin Lin Htet
Student@2024-05-16 11:30:54.491832
Bo Bo Min
Student@2024-05-16 11:31:46.663316
Bo Bo Min
Student@2024-05-16 11:32:46.591566
Bo Bo Min
Student@2024-05-28 13:42:15.710046
Bo Bo Min
Student@2024-05-28 13:42:26.078964
Bo Bo Min
Student@2024-05-28 13:44:56.223817
Bo Bo Min
Student@2024-05-28 13:45:06.209762
Bo Bo Min
Student@2024-05-28 13:47:28.067187
Bo Bo Min
Student@2024-05-28 13:47:37.311897
Bo Bo Min
Student@2024-05-28 13:48:37.206214
Bo Bo Min
Student@2024-05-28 13:48:47.472810
Bo Bo Min
Student@2024-05-28 13:49:49.864097
Bo Bo Min
Student@2024-05-28 13:49:59.757797
Bo Bo Min
Student@2024-05-28 13:50:10.152414
Bo Bo Min
Student@2024-05-28 13:54:32.147977
Bo Bo Min
Student@2024-05-28 13:56:12.214391
Bo Bo Min
Student@2024-05-28 13:56:22.539799
Bo Bo Min
Student@2024-05-28 13:56:32.872996
Bo Bo Min
Student@2024-05-28 13:57:15.771826
Bo B

In [34]:

# Define the name_role_to_batch dictionary
name_role_to_batch = {
    # 'Aung Khant Kyaw': 86,
    # 'Zin Lin Htet': 78,
    # 'Aung Paing Oo': 79,
    # 'Thin Yadanar Oo': 82,
    # 'Hsu Myat Noe': 86,
    # 'Tun Tun Nyein': 29,
    # 'Yin Kyi Phyu': 86,
    # 'Win Htut Naing': 82,
    # 'Su Lei Aung': 83,
    # 'Aye Myat Phoo Khine': 83,
    # "Ye' Thein Htut": 82,
    # 'Htan Lin Han': 82,
    # 'Wuttyi Oo': 86,
    # 'May Moe Pwint Phyu': 83,
    # 'Aye Chan Myat Phyoe': 86,
    # 'Dr. Moe Arkar Lwin': 'None'
    # Add all other mappings here
    'Bo Bo Min' : 78,
}

def update_attendance_logs_with_batch(name_role_to_batch):
    # Step 1: Retrieve the existing data from 'attendance:logs' key
    attendance_logs = r.lrange('attendance:logs', 0, -1)
    
    # Step 2: Modify each entry to include the batch information for students
    updated_logs = []
    for log in attendance_logs:
        log_str = log.decode()
        name_role, rest = log_str.split('@', 1)  # Split only once
        print(name_role)
        role, timestamp = rest.split('@', 1)  # Split again to get role and timestamp
        print(role)
        print(timestamp)
        if name_role in name_role_to_batch:
            log_str = f"{name_role}@{name_role_to_batch[name_role]}@{role}@{timestamp}"
        updated_logs.append(log_str)
    
    # Step 3: Update the modified entries back to the 'attendance:logs' key
    r.delete('attendance:logs')  # Clear the old data
    if updated_logs:
        r.lpush('attendance:logs', *updated_logs)

# Call the function to update the attendance logs with batch information
update_attendance_logs_with_batch(name_role_to_batch)


Bo Bo Min
Student
2024-05-13 13:22:28.389529
Bo Bo Min
Student
2024-05-16 11:29:28.776530
Zin Lin Htet
78
Student@2024-05-16 11:30:21.544281
Zin Lin Htet
78
Student@2024-05-16 11:30:54.491832
Bo Bo Min
Student
2024-05-16 11:31:46.663316
Bo Bo Min
Student
2024-05-16 11:32:46.591566
Bo Bo Min
Student
2024-05-28 13:42:15.710046
Bo Bo Min
Student
2024-05-28 13:42:26.078964
Bo Bo Min
Student
2024-05-28 13:44:56.223817
Bo Bo Min
Student
2024-05-28 13:45:06.209762
Bo Bo Min
Student
2024-05-28 13:47:28.067187
Bo Bo Min
Student
2024-05-28 13:47:37.311897
Bo Bo Min
Student
2024-05-28 13:48:37.206214
Bo Bo Min
Student
2024-05-28 13:48:47.472810
Bo Bo Min
Student
2024-05-28 13:49:49.864097
Bo Bo Min
Student
2024-05-28 13:49:59.757797
Bo Bo Min
Student
2024-05-28 13:50:10.152414
Bo Bo Min
Student
2024-05-28 13:54:32.147977
Bo Bo Min
Student
2024-05-28 13:56:12.214391
Bo Bo Min
Student
2024-05-28 13:56:22.539799
Bo Bo Min
Student
2024-05-28 13:56:32.872996
Bo Bo Min
Student
2024-05-28 13:57:15.77182